In [22]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from ladybug.sql import SQLiteResult
from ladybug.analysisperiod import AnalysisPeriod

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px

from outputs.sql import SQLReader

from geomeppy import IDF
from geometry.geometry_parser import GeometryParser
from outputs.output_data import GeometryOutputData

from helpers.plots import plot_polygon, plot_line_string, get_plottable_coords

In [2]:
sql = SQLReader("test03")

In [3]:
sql.match_geom_sql()

In [4]:
sql.dataset_names

['zone_mean_air_temp12-21', 'zone_mean_air_temp7-21']

In [40]:
coords = [c for c in sql.zone_dict["BLOCK NORTH STOREY 0"].polygon.exterior.coords]
coords[0]


class RectFromCoords:
    # ref: https://plotly.com/python/reference/layout/shapes/#layout-shapes-items-shape-x0
    def __init__(self, coords) -> None:
        xs = [i[0]for i in coords]
        ys = [i[1]for i in coords]
        self.x0 = min(xs)
        self.x1 = max(xs)
        self.y0 = min(ys)
        self.y1 = max(ys)

    def __repr__(self):
        return f"Rect(({self.x0}, {self.y0}), ({self.x1}, {self.y1}))"


In [17]:
import plotly.graph_objects as go

In [41]:
r = RectFromCoords(coords)
r

Rect((0.0, 7.0), (10.0, 11.0))

In [37]:
fig = go.Figure()
fig.add_shape(
    type="rect",
    xref="x", yref="y",
    fillcolor='turquoise',
    x0=2.5, y0=0,
    x1=3.5, y1=2,
    label=dict(text="Text in rectangle")
)
fig.show()

In [42]:
r = RectFromCoords(coords)
print(r)


fig = go.Figure()
fig.add_shape(
    type="rect",
    xref="x", yref="y",
    fillcolor='turquoise',
    x0=r.x0,
    y0=r.y0,
    x1=r.x1,
    y1=r.y1,
    label=dict(text="Text in rectangle")
)
fig.show()

Rect((0.0, 7.0), (10.0, 11.0))


In [27]:
coords = get_plottable_coords(sql.zone_dict["BLOCK NORTH STOREY 0"].polygon.exterior.coords)
x,y = coords 


fig = go.Figure()
color = "blue"
t = go.Scatter(
        x=x,
        y=y,
        fill="toself",
        marker=dict(color=color),
        fillcolor=color, 
        opacity=0.5,
        line_width=0,
        name="test",
)
fig.add_trace(t)
fig.show()

In [5]:
dataset = sql.dataset_names[0]

sql.extract_time_data(12, dataset)
sql.prepare_spatial_colors(dataset)

In [6]:
sql.zone_dict["BLOCK WEST STOREY 0"].extracted_data["zone_mean_air_temp12-21"].color

'rgb(122, 4, 2)'

In [7]:
traces = []

for zone in sql.zone_dict.values():
    val = round(zone.extracted_data[dataset].value,2)
    color = zone.extracted_data[dataset].color
    trace = plot_polygon(zone.polygon, color=color, label=f"{zone.name} - {val}ºC")
    traces.append(trace)

    for wall in zone.walls:
        trace = plot_line_string(wall.line, color="black", label=f"Wall {wall.number}")
        traces.append(trace)



In [8]:
fig = go.Figure()
for t in traces:
    fig.add_trace(t)

fig['layout']['showlegend'] = False
fig.add_trace(sql.colorbar_trace)
fig.show()

In [9]:
import numpy as np

In [10]:
np.arange(-11.3,-11.0, 0.05)

array([-11.3 , -11.25, -11.2 , -11.15, -11.1 , -11.05, -11.  ])